In [1]:
import pandas as pd
import pyspark.pandas as ps
import requests
from pca import pca
import json

from sqlalchemy import create_engine

from pyspark.sql.functions import pandas_udf, PandasUDFType, max, col, countDistinct, when, rank, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import ClusterCentroids


# from backend_spark.doris_common.doris_client import DorisClient
from functools import reduce
from pyspark.sql import DataFrame
from typing import Optional

In [2]:
df1_pandas = pd.read_csv("small2.csv")
df1_pandas

,TOOL_ID,TOOL_NAME,RUN_ID,EQP_NAME,CASE_INFO,PRODUCT_ID,PRODG1,OPER_NO,LOT_ID,WAFER_ID,...,LOWER_OUTLIER,UPPER_OUTLIER,RULES_ENABLED,ALARM_RULE,RESULT,STATUS,REGION,ERROR_MSG,STATISTIC_RESULT,VERSION
0,9289,PBT01_CGHA_4-34,351230,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-05,...,NaN,NaN,1,NaN,249.984800,NORMAL,LOWER_NORMAL,NaN,249.984800,3
1,9287,PBT01_CGHA_4-14,360329,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-06,...,NaN,NaN,1,NaN,249.984583,NORMAL,LOWER_NORMAL,NaN,249.984583,3
2,9279,PBT01_CLHA_4-12,360246,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-06,...,NaN,NaN,1,NaN,150.007500,NORMAL,LOWER_NORMAL,NaN,150.007500,4
3,9287,PBT01_CGHA_4-14,329563,PBT01,2023-10-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX221.100,NBX221-13,...,NaN,NaN,1,NaN,249.986250,NORMAL,LOWER_NORMAL,NaN,249.986250,3
4,9287,PBT01_CGHA_4-14,362271,PBT01,2023-12-20,AFPNR901N.0B0L,L2800Z2N,1V.PPB10,NBX293.200,NBX293-06,...,NaN,NaN,1,NaN,249.984583,NORMAL,LOWER_NORMAL,NaN,249.984583,3
5,9279,PBT01_CLHA_4-12,329480,PBT01,2023-10-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX221.100,NBX221-13,...,NaN,NaN,1,NaN,149.937083,NORMAL,LOWER_NORMAL,NaN,149.937083,4
6,9287,PBT01_CGHA_4-14,323876,PBT01,2023-09-28,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX220.150,NBX220-06,...,NaN,NaN,1,NaN,249.982500,NORMAL,LOWER_NORMAL,NaN,249.982500,3
7,9280,PBT01_CLHA_4-21,317196,PBT01,2023-09-25,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX219.130,NBX219-08,...,NaN,NaN,1,NaN,149.998261,NORMAL,LOWER_NORMAL,NaN,149.998261,4
8,9288,PBT01_CGHA_4-24,312691,PBT01,2023-09-13,AFPNR901N.0B0J,L2800Z2N,1V.PPB10,NBX220.040,NBX220-20,...,NaN,NaN,1,NaN,249.988750,NORMAL,UPPER_NORMAL,NaN,249.988750,3
9,9288,PBT01_CGHA_4-24,355016,PBT01,2023-12-16,AFPZM801N.0A01,L2800Z3N,1V.PPB10,NBX265.000,NBX265-08,...,NaN,NaN,1,NaN,249.982000,NORMAL,LOWER_NORMAL,NaN,249.982000,3


In [125]:
# df1_pandas['WAFER_ID'].unique()
# df1_pandas['PRODG1'].unique()

In [3]:
df1 = ps.from_pandas(df1_pandas).to_spark()

/usr/local/spark/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
df1.count()

24

In [5]:
############################################
######## 1. 客户只定义了bad_wafer = []是什么  ########
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# F.col('WAFER_ID').like('NDJ065%') | F.col('WAFER_ID').like('NDJ067%') 作为条件传入增加label
# 同时将isin模式也作为条件传入增加label

def get_label_single(df, bad_wafer):
    like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_like_conditions = " | ".join(like_conditions)
    isin_conditions = "col('WAFER_ID').isin(bad_wafer)"
    df = df.withColumn('label', 
                when( eval(all_like_conditions) | eval(isin_conditions), int(1)).otherwise(int(0)))
    return df


############################################
## 2. 客户定义了bad_wafer = [] 和 good_wafer = []######
############################################
# 将传进来的BAD_WAFER, 用 | 连接起来，
# 将传进来的GOOD_WAFER, 也用 | 连接起来，
# 同时将isin模式也作为条件传入增加label

def get_label_double(df, bad_wafer, good_wafer):
    good_like_conditions = [f"col('WAFER_ID').like('{good}')" for good in good_wafer]
    all_good_like_conditions = " | ".join(good_like_conditions)
    good_isin_conditions = "col('WAFER_ID').isin(good_wafer)"

    bad_like_conditions = [f"col('WAFER_ID').like('{bad}')" for bad in bad_wafer]
    all_bad_like_conditions = " | ".join(bad_like_conditions)
    bad_isin_conditions = "col('WAFER_ID').isin(bad_wafer)"

    df = df.withColumn('label',  when(eval(all_good_like_conditions) | eval(good_isin_conditions), int(0)).when(eval(all_bad_like_conditions) | eval(bad_isin_conditions), int(1)).otherwise(222333))
    df = df.filter(df['label'] != int(222333))
    return df

In [6]:
# small1.csv
# good = ["NBX221-13","NBX220-06","NBX272-19","NBX219-08"]
# bad  = ["NBX265-05","NBX265-06","NBX265-07","NBX265-08",
#         "NBX293-06","NBX293-07","NAZ998-21","NBX220-20"]

# df1 = get_label_double(df1, bad, good)

# small2.csv
good = ["NBX221-13","NBX220-06","NBX272-19","NBX219-08"]
bad  = ["NBX265-05","NBX265-06","NBX265-07","NBX265-08","NBX293-06","NBX293-07","NAZ998-21","NBX220-20"]

df1 = get_label_double(df1, bad, good)

In [7]:
df1.count()

24

In [8]:
bad_wafer_num = df1.filter("label == 1").select('WAFER_ID').distinct().count()
good_wafer_num = df1.filter("label == 0").select('WAFER_ID').distinct().count()
print(bad_wafer_num)
print(good_wafer_num)

8
4


In [9]:
############################################################################
##################################FDC数据预处理###############################
############################################################################
def _pre_process(df):
    """
    param df: 从数据库中读取出来的某个CASE数据
    return: 数据预处理，后面要根据实际情况统一添加
    """
    try:
        # 只选出会用到的列
        df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 'TOOL_NAME',
                       'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
        # 剔除NA值
        df = df.filter(col('STATISTIC_RESULT').isNotNull())
        # 按照所有的行进行去重
        df1 = df.dropDuplicates()
        # 选最新的RUN
        df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
        df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                          on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
        return df_run
    except Exception as e:
        return None



def commonality_analysis(df_run, grpby_list):
    """
    param df_run: 数据预处理后的数据
    return: 共性分析后的结果， 返回bad wafer前十的组合
    """
    try:
        grps = (df_run.groupBy(grpby_list)
                .agg(countDistinct('WAFER_ID').alias('wafer_count'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                .orderBy('bad_num', ascending=False))

        # 单站点+单腔室的情况
        if grps.count() == 1:
            return grps
        else:
            grps = grps.filter(grps['bad_num'] > 0)
            window_sep = Window().orderBy(col("bad_num").desc())
            ranked_df = grps.withColumn("rank", rank().over(window_sep))
            grpss = ranked_df.filter(col("rank") <= 10).drop("rank")
            return grpss
    except Exception as e:
        return None


# common_res = commonality_analysis(df_run)

In [10]:
df_run = _pre_process(df1)
print(df_run.count())

24


In [176]:
grpby_list = ['OPER_NO', 'TOOL_NAME']
common_res = commonality_analysis(df_run, grpby_list)
common_res.show()

+--------+---------------+-----------+--------+-------+
| OPER_NO|      TOOL_NAME|wafer_count|good_num|bad_num|
+--------+---------------+-----------+--------+-------+
|1V.PPB10|PBT01_CGHA_4-14|          6|       3|      3|
|1V.PPB10|PBT01_CLHA_4-12|          6|       3|      3|
|1V.PPB10|PBT01_CGHA_4-24|          3|       1|      2|
|1V.PPB10|PBT01_CLHA_4-21|          3|       1|      2|
|1V.PPB10|PBT01_CLHA_4-11|          2|       0|      2|
|1V.PPB10|PBT01_CGHA_4-13|          2|       0|      2|
|1V.PPB10|PBT01_CLHA_4-31|          1|       0|      1|
|1V.PPB10|PBT01_CGHA_4-34|          1|       0|      1|
+--------+---------------+-----------+--------+-------+



In [177]:
############################################################################
#################################获取样本数据#################################
############################################################################
def get_data_list(common_res, grpby_list, big_or_small='big'):
    """
    param common_res: 共性分析后的结果, 按照大样本或者小样本条件筛选出组合
    param grpby_list: 按照PRODG1+OPER_NO+TOOL_NAME分组或OPER_NO+TOOL_NAME分组
    param big_or_small: big或者small
    return: 对应组合的字典形式, 包在一个大列表中
    """
    assert big_or_small in ['big', 'small'], "只能选择big或者small, 请检查拼写"
    try:
        if big_or_small == 'big':
            good_bad_grps = common_res.filter("good_num >= 3 AND bad_num >= 3")
        else:
            good_bad_grps = common_res.filter("bad_num >= 1 AND wafer_count >=2")
        good_bad_grps = good_bad_grps.orderBy(col("bad_num").desc(), col("wafer_count").desc(), col("good_num").desc()).limit(5)
        
        if 'PRODG1' in grpby_list:
            data_list = good_bad_grps['PRODG1', 'OPER_NO', 'TOOL_NAME'].collect()  
        else:
            data_list = good_bad_grps['OPER_NO', 'TOOL_NAME'].collect()
            
        data_dict_list = [row.asDict() for row in data_list]
        return data_dict_list
    except Exception as e:
        return None


def get_train_data(df_run, data_dict_list):
    """
    param df_run: 数据预处理后的数据
    param data_dict: 筛选后的字典结果
    return: 从原始数据中过滤出真正用来建模的组合数据
    """
    try:
        if len(data_dict_list[0]) == 3:
            prod, oper, tool = data_dict_list[0]['PRODG1'], data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
            df_s = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' AND TOOL_NAME == '{}'".format(prod, oper, tool))
            for i in range(1, len(data_dict_list)):
                prod, oper, tool = data_dict_list[i]['PRODG1'], data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
                df_m = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' and TOOL_NAME == '{}'".format(prod, oper, tool))
                df_s = df_s.union(df_m)
        else:
            oper, tool = data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
            df_s = df_run.filter("OPER_NO == '{}' AND TOOL_NAME == '{}'".format(oper, tool))
            for i in range(1, len(data_dict_list)):
                oper, tool = data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
                df_m = df_run.filter("OPER_NO == '{}' and TOOL_NAME == '{}'".format(oper, tool))
                df_s = df_s.union(df_m)
        return df_s
    except Exception as e:
        return None

In [198]:
data_dict_list_ss = get_data_list(common_res=common_res, grpby_list=grpby_list, big_or_small='small')
data_dict_list_ss

[{'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CGHA_4-14'},
 {'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CLHA_4-12'},
 {'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CGHA_4-24'},
 {'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CLHA_4-21'},
 {'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CLHA_4-11'}]

In [199]:
df_run_ss = get_train_data(df_run=df_run, data_dict_list=data_dict_list_ss)
print(df_run_ss.count())

20


In [200]:
############################################################################
#########################获取传入的整个数据中的所有bad_wafer个数############
############################################################################
def get_all_bad_wafer_num(df):
    """
    param df: 筛选后的数据
    return: 数据中所有bad_wafer的数量
    """
    return df.filter("label == 1").select('WAFER_ID').distinct().count()

In [201]:
bad_wafer_num_small_sample = get_all_bad_wafer_num(df_run_ss)
bad_wafer_num_small_sample

7

------

In [113]:
df_original_pandas = df_run.toPandas()

In [100]:
model_res_ss.toPandas()

,PRODG1,OPER_NO,TOOL_NAME,parametric_name,weight,stats
0,grplen2,1V.EEU10,EUT0B_PM6,GAS11FLOW#AUTO_WINDOWS,0.259844,SUM
1,grplen2,1V.EEU10,EUT0B_PM6,GAS13FLOW#AUTO_WINDOWS,0.259709,SUM
2,grplen2,2U.CMT10,MTA03_4,KITS_LIFE#LIFE,0.170431,MAX
3,grplen2,2U.CDG20,DGA01_A1,TOTAL_HE_C#TOTAL_GAS,0.120509,MAX
4,grplen2,2U.CDG10,DGA03_C2,TOTAL_O2#TOTAL_GAS,0.102911,MAX
5,grplen2,2U.WSC20,SCT07_5-1,CHUCK_ROTATION_SPEED#MEAN,0.086596,MEAN


In [119]:
df_pa = df_run_pandas.query("OPER_NO == '{}' & TOOL_NAME == '{}'".format('2U.WSC20', 'SCT07_5-1'))
df_param = df_pa[df_pa['parametric_name'].str.contains("CHUCK_ROTATION_SPEED#MEAN")]
df_param

,WAFER_ID,OPER_NO,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,parametric_name,STATISTIC_RESULT,label
467,NBX265-08,2U.WSC20,6743,1866474,SCT07,AFPZM801N.0A01,L2800Z3N,SCT07_5-1,CHUCK_ROTATION_SPEED#MEAN#MEAN,1360.093750,1
480,NBX265-06,2U.WSC20,6743,1866475,SCT07,AFPZM801N.0A01,L2800Z3N,SCT07_5-1,CHUCK_ROTATION_SPEED#MEAN#MEAN,1363.935484,1


In [120]:
df_pa = df_original_pandas.query("OPER_NO == '{}' & TOOL_NAME == '{}'".format('2U.WSC20', 'SCT07_5-1'))
df_param = df_pa[df_pa['parametric_name'].str.contains("CHUCK_ROTATION_SPEED#MEAN")]
df_param

,WAFER_ID,OPER_NO,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,parametric_name,STATISTIC_RESULT,label
2029,NBX265-08,2U.WSC20,6743,1866474,SCT07,AFPZM801N.0A01,L2800Z3N,SCT07_5-1,CHUCK_ROTATION_SPEED#MEAN#MEAN,1360.093750,1
2688,NBX265-06,2U.WSC20,6743,1866475,SCT07,AFPZM801N.0A01,L2800Z3N,SCT07_5-1,CHUCK_ROTATION_SPEED#MEAN#MEAN,1363.935484,1


---

In [132]:
df_run_pandas = df_run_ss.toPandas()
df_run_pandas

,WAFER_ID,OPER_NO,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,parametric_name,STATISTIC_RESULT,label
0,NBX221-13,1V.PPB10,9287,329563,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.986250,0
1,NBX293-06,1V.PPB10,9287,362271,PBT01,AFPNR901N.0B0L,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.984583,1
2,NBX220-06,1V.PPB10,9287,323876,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.982500,0
3,NBX272-19,1V.PPB10,9287,331771,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.985417,0
4,NBX221-13,1V.PPB10,9279,329480,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CLHA_4-12,PLATE_TEMP#DHP150_MEAN#MEAN,149.937083,0
5,NBX272-19,1V.PPB10,9279,331688,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CLHA_4-12,PLATE_TEMP#DHP150_MEAN#MEAN,150.034583,0
6,NBX293-06,1V.PPB10,9279,362188,PBT01,AFPNR901N.0B0L,L2800Z2N,PBT01_CLHA_4-12,PLATE_TEMP#DHP150_MEAN#MEAN,150.013750,1
7,NBX220-06,1V.PPB10,9279,323793,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CLHA_4-12,PLATE_TEMP#DHP150_MEAN#MEAN,150.022917,0
8,NBX219-08,1V.PPB10,9280,317196,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CLHA_4-21,PLATE_TEMP#DHP150_MEAN#MEAN,149.998261,0
9,NBX220-20,1V.PPB10,9280,312595,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CLHA_4-21,PLATE_TEMP#DHP150_MEAN#MEAN,150.010833,1


In [133]:
data_dict_list_ss

[{'PRODG1': 'L2800Z2N', 'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CGHA_4-14'},
 {'PRODG1': 'L2800Z2N', 'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CLHA_4-12'},
 {'PRODG1': 'L2800Z2N', 'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CLHA_4-21'},
 {'PRODG1': 'L2800Z2N', 'OPER_NO': '1V.PPB10', 'TOOL_NAME': 'PBT01_CGHA_4-24'}]

In [134]:
i = 0
prod, oper, tool = data_dict_list_ss[i]['PRODG1'], data_dict_list_ss[i]['OPER_NO'], data_dict_list_ss[i]['TOOL_NAME']
df_run_pandas1 = df_run_pandas.query("PRODG1 == '{}' & OPER_NO == '{}' & TOOL_NAME == '{}'".format(prod, oper, tool))
df_run_pandas1

,WAFER_ID,OPER_NO,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,parametric_name,STATISTIC_RESULT,label
0,NBX221-13,1V.PPB10,9287,329563,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.986250,0
1,NBX293-06,1V.PPB10,9287,362271,PBT01,AFPNR901N.0B0L,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.984583,1
2,NBX220-06,1V.PPB10,9287,323876,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.982500,0
3,NBX272-19,1V.PPB10,9287,331771,PBT01,AFPNR901N.0B0J,L2800Z2N,PBT01_CGHA_4-14,PLATE_TEMP#HDB205C60S_MEAN#MEAN,249.985417,0


In [135]:
df_pivot = df_run_pandas1.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                         columns=['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name'],
                                                         values=['STATISTIC_RESULT'])
df_pivot.columns = df_pivot.columns.map('#'.join)
df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
df_pivot

,WAFER_ID,label,STATISTIC_RESULT#L2800Z2N#1V.PPB10#PBT01_CGHA_4-14#PLATE_TEMP#HDB205C60S_MEAN#MEAN
0,NBX220-06,0,249.982500
1,NBX221-13,0,249.986250
2,NBX272-19,0,249.985417
3,NBX293-06,1,249.984583


In [136]:
# 定义自变量
x_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]


# 建立模型
model = pca(n_components=0.8, verbose=None)
results = model.fit_transform(x_train)
res_top = results['topfeat']
res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
res_top_select

,feature,loading
0,STATISTIC_RESULT#L2800Z2N#1V.PPB10#PBT01_CGHA_...,-1.0


In [112]:
res_top_select['importance'] = abs(res_top_select['loading'])
res_top_select = res_top_select.drop("loading", axis=1)
res_top_select

,features,importance
0,STATISTIC_RESULT#L2800Z2N#1V.PPB10#PBT01_CGHA_...,1.0


In [113]:
res_top_select['bad_wafer'] = sum(df_pivot['label'])
res_top_select['OPER_NO'] = df_run_pandas1['OPER_NO'].values[0]
res_top_select['TOOL_NAME'] = df_run_pandas1['TOOL_NAME'].values[0]
if len(grpby_list) == 3: 
    res_top_select['PRODG1'] = df_run_pandas1['PRODG1'].values[0]
else:
    res_top_select['PRODG1'] = 'grplen2'

In [115]:
res_top_select

,features,importance,bad_wafer,OPER_NO,TOOL_NAME,PRODG1
0,STATISTIC_RESULT#L2800Z2N#1V.PPB10#PBT01_CGHA_...,1.0,1,1V.PPB10,PBT01_CGHA_4-24,L2800Z2N


In [98]:
res_top_select['feature'].iloc[0]

'STATISTIC_RESULT#L2800Z2N#1V.PPB10#PBT01_CGHA_4-14#PLATE_TEMP#HDB205C60S_MEAN#MEAN'

In [77]:
# 0. 'STATISTIC_RESULT#L2800Z3N#2U.CMT10#MTA03_4#KITS_LIFE#LIFE#MAX'  0.707122
# 1. 'STATISTIC_RESULT#L2800Z3N#1V.EEU10#EUT0B_PM6#GAS11FLOW#AUTO_WINDOWS#SUM'  0.969570
# 1. 'STATISTIC_RESULT#L2800Z3N#1V.EEU10#EUT0B_PM6#GAS13FLOW#AUTO_WINDOWS#SUM'  0.966666
# 2. 'STATISTIC_RESULT#L2800Z3N#2U.CDB10#DBA71_A1#HE_MDEOS#TRANS#MEAN' 0.629773
# 3. 'STATISTIC_RESULT#L2800Z3N#2U.WSC10#SCT07_4-3#CHUCK_ROTATION_SPEED#MEAN#MEAN'  0.829139
# 4. 'STATISTIC_RESULT#L2800Z3N#2U.WSC50#SCT06_5-4#CHUCK_ROTATION_SPEED#MEAN#MEAN'  0.68266

---

In [205]:
def get_pivot_table(df, by):
    if len(by) == 3:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                     columns=['OPER_NO', 'TOOL_NAME', 'parametric_name', 'PRODG1'],
                                                     values=['STATISTIC_RESULT'])
    else:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                     columns=['OPER_NO', 'TOOL_NAME', 'parametric_name'],
                                                     values=['STATISTIC_RESULT'])
    df_pivot.columns = df_pivot.columns.map('#'.join)
    df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
    return df_pivot

In [206]:
def fit_pca_small_sample(df, by):
    """
    param df: 小样本组合的数据
    param by: 分组字段
    return: PCA建模后的结果
    """
    try:
        schema_all = StructType([StructField("PRODG1", StringType(), True),
                                 StructField("OPER_NO", StringType(), True),
                                 StructField("TOOL_NAME", StringType(), True),
                                 StructField("features", StringType(), True),
                                 StructField("importance", FloatType(), True),
                                 StructField("bad_wafer", IntegerType(), True)]) 

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            df_pivot = get_pivot_table(df=df_run, by=by)
            
            # 定义自变量
            x_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]

            # 建立模型
            model = pca(n_components=0.8, verbose=None)
            results = model.fit_transform(x_train)
            res_top = results['topfeat']
            res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
            res_top_select['importance'] = abs(res_top_select['loading'])
            res_top_select = res_top_select.rename(columns={'feature': 'features'})
            res_top_select = res_top_select.drop("loading", axis=1)

            # 增加一些字段信息
            res_top_select['bad_wafer'] = sum(df_pivot['label'])
            res_top_select['OPER_NO'] = df_run['OPER_NO'].values[0]
            res_top_select['TOOL_NAME'] = df_run['TOOL_NAME'].values[0]
            if len(by) == 3: 
                res_top_select['PRODG1'] = df_run['PRODG1'].values[0]
            else:
                res_top_select['PRODG1'] = 'grplen2'
         
            return res_top_select
        return df.groupby(by).apply(get_model_result)
    except Exception as e:
        return None

In [207]:
res = fit_pca_small_sample(df=df_run_ss, by=grpby_list)
print(grpby_list)
res.count()

['OPER_NO', 'TOOL_NAME']


5

In [208]:
res.show()

+-------+--------+---------------+--------------------+----------+---------+
| PRODG1| OPER_NO|      TOOL_NAME|            features|importance|bad_wafer|
+-------+--------+---------------+--------------------+----------+---------+
|grplen2|1V.PPB10|PBT01_CGHA_4-14|STATISTIC_RESULT#...|       1.0|        3|
|grplen2|1V.PPB10|PBT01_CGHA_4-24|STATISTIC_RESULT#...|       1.0|        2|
|grplen2|1V.PPB10|PBT01_CLHA_4-11|STATISTIC_RESULT#...|       1.0|        2|
|grplen2|1V.PPB10|PBT01_CLHA_4-12|STATISTIC_RESULT#...|       1.0|        3|
|grplen2|1V.PPB10|PBT01_CLHA_4-21|STATISTIC_RESULT#...|       1.0|        2|
+-------+--------+---------------+--------------------+----------+---------+



In [209]:
res.toPandas()[['features']].iloc[0].values

array(['STATISTIC_RESULT#1V.PPB10#PBT01_CGHA_4-14#PLATE_TEMP#HDB205C60S_MEAN#MEAN'],
      dtype=object)

In [185]:
def split_features(df, index) -> str:
    """
    param df: RandomForest建模后的feature_importance_table
    param index: 顺序值
    return: 字段属性值
    """
    try:
        return df['features'].apply(lambda x: x.split('#')[index])
    except Exception as e:
        return None


def get_split_feature_importance_table(df, by):
    """
    param df: RandomForest建模后的feature_importance_table
    param by: OPER_NO+TOOL_NAME+PRODG1或者OPER_NO+TOOL_NAME
    return: 分裂features后的表
    """
    try:
        df['STATISTIC_RESULT'] = split_features(df, 0)
        df['OPER_NO'] = split_features(df, 1)
        df['TOOL_NAME'] = split_features(df, 2)
        df['parametric_name'] = split_features(df, 3)
        df['step'] = split_features(df, 4)
        df['stats'] = split_features(df, 5)

        if 'PRODG1' in by:
            df['PRODG1'] = split_features(df, 6)
        else:
            df = df.assign(PRODG1 = 'grplen2')

        df = df.drop(['features', 'STATISTIC_RESULT'], axis=1).reset_index(drop=True)
        return df
    except Exception as e:
        return None


def add_feature_stats(df):
    """
    param df: 经过处理后的feature_importance_table
    return: 新增一列，含有参数的所有统计特征:feature_stats
    """
    try:
        feature_stats = df.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name', 'step'])['stats'].unique().reset_index()
        feature_stats['stats'] = [feature_stats['stats'].iloc[i].tolist() for i in range(len(feature_stats))]
        feature_stats['stats'] = feature_stats['stats'].apply(lambda x: "#".join(x))
        feature_stats = feature_stats.assign(parametric_name=lambda x: x['parametric_name']+str('#')+x['step']).drop('step', axis=1)
        return feature_stats
    except Exception as e:
        return None

In [186]:
def split_calculate_features_small_sample(df, by):
    """
    param df: PCA建模后的结果
    param by: 分组字段
    return: features和importance结果
    """
    try:
        schema_all = StructType([ StructField("PRODG1", StringType(), True),
                                  StructField("OPER_NO", StringType(), True),
                                  StructField("TOOL_NAME", StringType(), True),
                                  StructField("parametric_name", StringType(), True),
                                  StructField("importance", FloatType(), True),
                                  StructField("bad_wafer", FloatType(), True),
                                  StructField("stats", StringType(), True)]) 

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(model_results):        
            feature_importance_table = model_results[['features',  'importance', 'bad_wafer']].dropna(axis=0)
            # 分裂features
            feature_importance_res_split = get_split_feature_importance_table(feature_importance_table, by)

            # 新增一列，含有参数的所有统计特征:feature_stats
            feature_stats = add_feature_stats(feature_importance_res_split)

            #对同一种组合里的同一个参数进行求和:feature_importance_groupby
            feature_importance_groupby = (feature_importance_res_split.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME', 'bad_wafer',
                                                                             'parametric_name','step'])['importance'].sum().reset_index())
            feature_importance_groupby = feature_importance_groupby.assign(parametric_name=lambda x: x['parametric_name']+str('#')+x['step']).drop('step', axis=1)

            # feature_stats和feature_importance_groupby连接
            grpby_stats = pd.merge(feature_stats, feature_importance_groupby, on=['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name']).dropna().reset_index(drop=True)
            return grpby_stats
        return df.groupby(by).apply(get_result)
    except Exception as e:
        return None

In [187]:
f_res = split_calculate_features_small_sample(df=res, by=grpby_list)
f_res.show()

+-------+--------+---------------+--------------------+----------+---------+-----+
| PRODG1| OPER_NO|      TOOL_NAME|     parametric_name|importance|bad_wafer|stats|
+-------+--------+---------------+--------------------+----------+---------+-----+
|grplen2|1V.PPB10|PBT01_CGHA_4-14|PLATE_TEMP#HDB205...|       1.0|      3.0| MEAN|
|grplen2|1V.PPB10|PBT01_CGHA_4-24|PLATE_TEMP#HDB205...|       1.0|      2.0| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-11|PLATE_TEMP#DHP150...|       1.0|      2.0| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-12|PLATE_TEMP#DHP150...|       1.0|      3.0| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-21|PLATE_TEMP#DHP150...|       1.0|      2.0| MEAN|
+-------+--------+---------------+--------------------+----------+---------+-----+



In [188]:
# f_res.toPandas()

In [189]:
def get_finall_results_small_sample(f_res, bad_wafer_num):
    """
    param s_res: roc_auc分数结果
    param f_res: features和importance结果
    param bad_wafer_num: 数据中所有bad_wafer的数量
    return: 最后的建模结果
    """
    try:
        f_res = f_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)
        df_merge = f_res.withColumn('weight_original', col('importance') * col('bad_ratio'))

        # 最后再次进行一次归一化
        weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
        df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)

        df_merge = df_merge.select(['PRODG1', 'OPER_NO', 'TOOL_NAME',
                                    'parametric_name', 'weight', 'stats']).orderBy('weight', ascending=False)
        return df_merge
    except Exception as e:
        return None

In [190]:
bad_wafer_num_small_sample = get_all_bad_wafer_num(df_run_ss)
print(bad_wafer_num_small_sample)
model_res_ss = get_finall_results_small_sample(f_res=f_res, bad_wafer_num=bad_wafer_num_small_sample)

7


In [191]:
model_res_ss.show()

+-------+--------+---------------+--------------------+-------------------+-----+
| PRODG1| OPER_NO|      TOOL_NAME|     parametric_name|             weight|stats|
+-------+--------+---------------+--------------------+-------------------+-----+
|grplen2|1V.PPB10|PBT01_CGHA_4-14|PLATE_TEMP#HDB205...|0.25000000000000006| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-12|PLATE_TEMP#DHP150...|0.25000000000000006| MEAN|
|grplen2|1V.PPB10|PBT01_CGHA_4-24|PLATE_TEMP#HDB205...|0.16666666666666669| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-11|PLATE_TEMP#DHP150...|0.16666666666666669| MEAN|
|grplen2|1V.PPB10|PBT01_CLHA_4-21|PLATE_TEMP#DHP150...|0.16666666666666669| MEAN|
+-------+--------+---------------+--------------------+-------------------+-----+



In [145]:
#####################################################################################
#############################将建模后的结果增加特定的列####################################
#####################################################################################
def add_certain_column(df, by, request_id):
    """
    param df: 最后的建模结果
    param by: 分组字段, 手动增加一列add
    param request_id: 传入的request_id
    return: 最后的建模结果增加特定的列
    """
    try:
        schema_all = StructType([
            StructField("PRODG1", StringType(), True),
            StructField("OPER_NO", StringType(), True),
            StructField("TOOL_NAME", StringType(), True),
            StructField("stats", StringType(), True),
            StructField("parametric_name", StringType(), True),
            StructField("weight", FloatType(), True),
            StructField("request_id", StringType(), True),
            StructField("weight_percent", FloatType(), True),
            StructField("index_no", IntegerType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_result(final_res):
            final_res['weight'] = final_res['weight'].astype(float)
            final_res = final_res.query("weight > 0")
            final_res['request_id'] = request_id
            final_res['weight_percent'] = final_res['weight'] * 100
            final_res = final_res.sort_values('weight', ascending=False)
            final_res['index_no'] = [i + 1 for i in range(len(final_res))]
            final_res = final_res.drop('add', axis=1)
            final_res['PRODG1'] = final_res['PRODG1'].apply(lambda x: None if x == 'grplen2' else x)
            return final_res
        return df.groupby(by).apply(get_result)
    except Exception as e:
        return None

In [146]:
request_id = 'sdssd'
final_res_ss = model_res_ss.withColumn('add', lit(0))
final_res_add_columns = add_certain_column(df=final_res_ss, by='add', request_id=request_id)

In [147]:
final_res_add_columns.show()

+------+--------+---------------+-----+--------------------+----------+----------+--------------+--------+
|PRODG1| OPER_NO|      TOOL_NAME|stats|     parametric_name|    weight|request_id|weight_percent|index_no|
+------+--------+---------------+-----+--------------------+----------+----------+--------------+--------+
|  null|1V.PPB10|PBT01_CGHA_4-14| MEAN|PLATE_TEMP#HDB205...|      0.25|     sdssd|          25.0|       1|
|  null|1V.PPB10|PBT01_CLHA_4-12| MEAN|PLATE_TEMP#DHP150...|      0.25|     sdssd|          25.0|       2|
|  null|1V.PPB10|PBT01_CGHA_4-24| MEAN|PLATE_TEMP#HDB205...|0.16666667|     sdssd|     16.666666|       3|
|  null|1V.PPB10|PBT01_CLHA_4-11| MEAN|PLATE_TEMP#DHP150...|0.16666667|     sdssd|     16.666666|       4|
|  null|1V.PPB10|PBT01_CLHA_4-21| MEAN|PLATE_TEMP#DHP150...|0.16666667|     sdssd|     16.666666|       5|
+------+--------+---------------+-----+--------------------+----------+----------+--------------+--------+



In [148]:
final_res_add_columns.toPandas()

,PRODG1,OPER_NO,TOOL_NAME,stats,parametric_name,weight,request_id,weight_percent,index_no
0,None,1V.PPB10,PBT01_CGHA_4-14,MEAN,PLATE_TEMP#HDB205C60S_MEAN,0.250000,sdssd,25.000000,1
1,None,1V.PPB10,PBT01_CLHA_4-12,MEAN,PLATE_TEMP#DHP150_MEAN,0.250000,sdssd,25.000000,2
2,None,1V.PPB10,PBT01_CGHA_4-24,MEAN,PLATE_TEMP#HDB205C60S_MEAN,0.166667,sdssd,16.666666,3
3,None,1V.PPB10,PBT01_CLHA_4-11,MEAN,PLATE_TEMP#DHP150_MEAN,0.166667,sdssd,16.666666,4
4,None,1V.PPB10,PBT01_CLHA_4-21,MEAN,PLATE_TEMP#DHP150_MEAN,0.166667,sdssd,16.666666,5
